In [1]:
import modulo_conn_sql as mcq
import numpy as np
import pandas as pd 
import datetime 
import matplotlib.pyplot as plt
from pandas.tseries.offsets import MonthEnd
from pandas.tseries.offsets import MonthBegin
import random

def conectarSQL():
    conn = mcq.ConexionSQL()
    cursor = conn.getCursor()
    return cursor

#Query BD SQL-Server Cemex
def querySQL(query, parametros):
    #Conectar con base sql y ejecutar consulta
    cursor = conectarSQL()
    try:
        cursor.execute(query, parametros)
        #obtener nombre de columnas
        names = [ x[0] for x in cursor.description]
        
        #Reunir todos los resultado en rows
        rows = cursor.fetchall()
        resultadoSQL = []
            
        #Hacer un array con los resultados
        while rows:
            resultadoSQL.append(rows)
            if cursor.nextset():
                rows = cursor.fetchall()
            else:
                rows = None
                
        #Redimensionar el array para que quede en dos dimensiones
        resultadoSQL = np.array(resultadoSQL)
        resultadoSQL = np.reshape(resultadoSQL, (resultadoSQL.shape[1], resultadoSQL.shape[2]) )
    finally:
            if cursor is not None:
                cursor.close()
    return pd.DataFrame(resultadoSQL, columns = names)
 
# dataset(pandas dataframe): base de datos con el historico
# array_group_top(array) : array de la jerarquia mas ALTA con el nombre de las columnas del dataset por el cual se quiere agrupar las proporciones
# array_group_bottom(array) : array de la jerarquia mas BAJA con el nombre de las columnas del dataset por el cual se quiere agrupar las proporciones
# medida_target( string ) : nombre de la columna que contiene los datos objetivo de la proporcion
# group_target(array) : array de nombre de columnas con las cuales queremos la proporcion final
# name_proportion(string) : etiqueta de la dimension a la cual le estamos calculando la proporcion
    
def historical_proportion( dataset, array_group_top, array_group_bottom, medida_target, group_target, name_proportion  ):

    promedio_group_top = dataset.groupby(array_group_top)[medida_target].mean().reset_index()
    promedio_group_bottom = dataset.groupby(array_group_bottom)[medida_target].mean().reset_index()    
    proportion  = pd.merge(promedio_group_bottom, promedio_group_top, how = 'inner', left_on = array_group_top, right_on = array_group_top )
    proportion['proportion'] = proportion[medida_target + '_x'] / proportion[medida_target + '_y']
    proportion = proportion.groupby(group_target)['proportion'].mean().reset_index()
    proportion.rename(columns={'proportion':'proportion_' + name_proportion}, inplace = True)
    
    return proportion 

def random_number (num):
    
    num = num if num <= 1 else 1
    
    return 1 + random.uniform( 0, num) if  random.random() < 0.5 else 1 - random.uniform( 0, num) 

#desviacion estandar, media y coeficiente de variacion
def stats_serie(dataset, array_group, colum_target):
    ret = dataset.groupby(array_group)[colum_target].agg( ['std', 'mean']).reset_index()
    ret['cov'] = ret['std']/ret['mean']
    
    return ret
    

#PRINCIPAL

#años para entrenar las versiones
yearDesagregacion = 2021
mesDesagregacion = 12

#Targets
pais = 'Colombia'
volPais = 120840

nivel_volatilidad = 0.4

inicioHistoria = datetime.datetime(2021, 10, 1) #'2013-05-01'
finHistoria = datetime.datetime(2021, 11 , 11)

criterio_historia_reciente = 45 #dias
absorcionEstadistica = True #criterio para tomar las proporciones historicas de las plantas o False -> archivo que define el volumen por planta

#Consulta de datos en la base SQL
despachosSQL = querySQL(  "{CALL SCAC_AP8_BaseForecast (?,?,?)}", (pais, inicioHistoria.strftime("%Y-%m-%d"), finHistoria.strftime("%Y-%m-%d") ) )
calendarioLogistico = querySQL( "{CALL SCAC_AP9_CalendarioLogistico (?,?,?)}" , (pais, yearDesagregacion, mesDesagregacion))

nombre_cluster = querySQL( "SELECT Centro, Ciudad_Cluster as Ciudad, [Desc Cluster] as Cluster, [Planta Unica] as PlantaUnica FROM SCAC_AT1_NombreCluster where Pais = ?" , (pais) )

################################################################################################
irregularidades  = pd.read_excel("../datos/BaseIrregularidades.xlsx")
irregularidades['FechaEntrega'] = pd.to_datetime(irregularidades['FechaEntrega'])


In [62]:
despachosSQL['totalEntregado'] = despachosSQL['totalEntregado'].astype(float)
#irregularidades a nivel pais
irr_nacional = irregularidades[irregularidades['Ciudad']==pais]
despachos_irregularidades = pd.merge(despachosSQL, nombre_cluster, left_on='Planta', right_on='Centro')
despachos_irregularidades = pd.merge(despachos_irregularidades, irregularidades[['FechaEntrega','EtiquetaIrregularidad']], on='FechaEntrega', how='left')

In [65]:
proportion_irr = historical_proportion(despachos_irregularidades, ['Planta'], [ 'Planta', 'EtiquetaIrregularidad'], 'totalEntregado', ['Planta', 'EtiquetaIrregularidad'], 'irregularidad')
proportion_irr.head(20)

,Planta,EtiquetaIrregularidad,proportion_irregularidad
0,F001,martes post puente,0.950390
1,F001,sabado pre puente,0.900156
2,F001,viernes pre puente,1.097362
3,F003,martes post puente,1.100605
4,F003,sabado pre puente,1.011533
5,F003,viernes pre puente,1.132856
6,F006,martes post puente,0.949253
7,F006,sabado pre puente,0.984762
8,F006,viernes pre puente,1.082414
9,F007,martes post puente,1.031585


In [64]:
proportion_irr['EtiquetaIrregularidad'].unique()

array(['martes post puente', 'sabado pre puente', 'viernes pre puente'],
      dtype=object)

In [60]:
despachos_irregularidades.head(20)

,Año,Mes,Planta,TipoPlanta,totalEntregado,Semana_Relativa,DiaSemana,FechaEntrega,DiasOperativos,Centro,Ciudad,Cluster,PlantaUnica,EtiquetaIrregularidad
0,2021,10,F048,Central,297.75,5,3,2021-10-26,23,F048,Bogotá,CLUSTER CENTRO,CO-PLANTA PUENTE ARANDA,NaN
1,2021,10,F048,Central,271.5,3,5,2021-10-14,14,F048,Bogotá,CLUSTER CENTRO,CO-PLANTA PUENTE ARANDA,NaN
2,2021,10,F048,Central,342.5,2,3,2021-10-05,5,F048,Bogotá,CLUSTER CENTRO,CO-PLANTA PUENTE ARANDA,NaN
3,2021,10,F048,Central,326,3,4,2021-10-13,13,F048,Bogotá,CLUSTER CENTRO,CO-PLANTA PUENTE ARANDA,NaN
4,2021,10,F048,Central,250,2,7,2021-10-09,9,F048,Bogotá,CLUSTER CENTRO,CO-PLANTA PUENTE ARANDA,NaN
5,2021,10,F048,Central,14.75,3,1,2021-10-10,10,F048,Bogotá,CLUSTER CENTRO,CO-PLANTA PUENTE ARANDA,NaN
6,2021,10,F048,Central,184,1,7,2021-10-02,2,F048,Bogotá,CLUSTER CENTRO,CO-PLANTA PUENTE ARANDA,NaN
7,2021,10,F048,Central,233.5,5,4,2021-10-27,24,F048,Bogotá,CLUSTER CENTRO,CO-PLANTA PUENTE ARANDA,NaN
8,2021,10,F048,Central,264.5,4,3,2021-10-19,17,F048,Bogotá,CLUSTER CENTRO,CO-PLANTA PUENTE ARANDA,martes post puente
9,2021,11,F048,Central,267,1,4,2021-11-03,2,F048,Bogotá,CLUSTER CENTRO,CO-PLANTA PUENTE ARANDA,NaN


In [66]:
irr_nacional.head()

,Ciudad,FechaEntrega,EtiquetaIrregularidad
0,Colombia,2013-01-02,post año nuevo
1,Colombia,2013-01-05,viernes pre puente
2,Colombia,2013-01-06,sabado pre puente
3,Colombia,2013-01-08,martes post puente
13,Colombia,2013-03-22,viernes pre puente
